<a href="https://colab.research.google.com/github/namitakalra-google/Big-Data-Workshop/blob/main/labs/%5BSolution%5D_MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1>Introduction to Map Reduce</h1></center>

### Overview

1. Recap of functional programming in Python
2. Python's `map` and `reduce` functions
3. Writing parallel code using `map`
4. The Map-Reduce programming model

## Functional programming

Consider the following code:

In [ ]:
def double_everything_in(data):
    result = []
    for i in data:
        result.append(2 * i)
    return result

def quadruple_everything_in(data):
    result = []
    for i in data:
        result.append(4 * i)
    return result

In [ ]:
double_everything_in([1, 2, 3, 4, 5])

[2, 4, 6, 8, 10]

In [ ]:
quadruple_everything_in([1, 2, 3, 4, 5])

[4, 8, 12, 16, 20]

### DRY - Fundamental Programming Concept

- The above code violates the ["do not repeat yourself"](https://en.wikipedia.org/wiki/Don't_repeat_yourself_) principle of good software engineering practice.

- How can rewrite the code so that it avoids duplication?

In [ ]:
def multiply_by_x_everything_in(x, data):
    result = []
    for i in data:
        result.append(x * i)
    return result

In [ ]:
multiply_by_x_everything_in(2, [1, 2, 3, 4, 5])

[2, 4, 6, 8, 10]

In [ ]:
multiply_by_x_everything_in(4, [1, 2, 3, 4, 5])

[4, 8, 12, 16, 20]

- Now consider the following code:

In [ ]:
def squared(x):
    return x*x

def double(x):
    return x*2

def square_everything_in(data):
    result = []
    for i in data:
        result.append(squared(i))
    return result

def double_everything_in(data):
    result = []
    for i in data:
        result.append(double(i))
    return result

In [ ]:
square_everything_in([1, 2, 3, 4, 5])

[1, 4, 9, 16, 25]

In [ ]:
double_everything_in([1, 2, 3, 4, 5])

[2, 4, 6, 8, 10]

### DRY - Fundamental Programming Concept
- The above code violates the ["do not repeat yourself"](https://en.wikipedia.org/wiki/Don't_repeat_yourself_) principle of good software engineering practice.

- How can rewrite the code so that it avoids duplication?

### Passing Functions as Values
- Functions can be passed to other functions as values.
-


In [ ]:
def apply_f_to_everything_in(f, data):
    result = []
    for x in data:
        result.append(f(x))
    return result

In [ ]:
apply_f_to_everything_in(squared, [1, 2, 3, 4, 5])

[1, 4, 9, 16, 25]

In [ ]:
apply_f_to_everything_in(double, [1, 2, 3, 4, 5])

[2, 4, 6, 8, 10]

### Lambda expressions

- We can use anonymous functions to save having to define a function each time we want to use map.

In [ ]:
apply_f_to_everything_in(lambda x: x*x, [1, 2, 3, 4, 5])

[1, 4, 9, 16, 25]

# Python's `map` function

- Python has a built-in function `map` which is much faster than our version.



In [ ]:
map(lambda x: x*x, [1, 2, 3, 4, 5])

In [ ]:
list(map(lambda x: x*x, [1, 2, 3, 4, 5]))

[1, 4, 9, 16, 25]

## Implementing reduce

- The `reduce` function is an example of a [fold](https://en.wikipedia.org/wiki/Fold_%28higher-order_function%29).

- There are different ways we can fold data.

- The following implements a *left* fold.


In [ ]:
def foldl(f, data, z):
    if (len(data) == 0):
        print (z)
        return z
    else:
        head = data[0]
        tail = data[1:]
        print ("Folding", head, "with", tail, "using", z)
        partial_result = f(z, data[0])
        print ("Partial result is", partial_result)
        return foldl(f, tail, partial_result)

In [ ]:
foldl(lambda x, y: x + y, [1, 2, 3, 4, 5], 0)

Folding 1 with [2, 3, 4, 5] using 0
Partial result is 1
Folding 2 with [3, 4, 5] using 1
Partial result is 3
Folding 3 with [4, 5] using 3
Partial result is 6
Folding 4 with [5] using 6
Partial result is 10
Folding 5 with [] using 10
Partial result is 15
15


15

## Python's `reduce` function.

- Python's built-in `reduce` function is a *left* fold.

In [ ]:
from functools import reduce
reduce(lambda x, y: x + y, [1, 2, 3, 4, 5])

15

# Functional programming and parallelism

- Functional programming lends itself to [parallel programming](https://computing.llnl.gov/tutorials/parallel_comp/#Models).

- The `map` function can easily be parallelised through [data-level parallelism](https://en.wikipedia.org/wiki/Data_parallelism),
    - provided that the function we supply as an argument is *free from* [side-effects](https://en.wikipedia.org/wiki/Side_effect_%28computer_science%29)
        - (which is why we avoid working with mutable data).

- We can see this by rewriting it so:


In [ ]:
def perform_computation(f, result, data, i):
    print ("Computing the ", i, "th result...")
    # This could be scheduled on a different CPU
    result[i] = f(data[i])

def my_map(f, data):
    result = [None] * len(data)
    for i in range(len(data)):
        perform_computation(f, result, data, i)
    # Wait for other CPUs to finish, and then..
    return result

In [ ]:
my_map(lambda x: x * x, [1, 2, 3, 4, 5])

Computing the  0 th result...
Computing the  1 th result...
Computing the  2 th result...
Computing the  3 th result...
Computing the  4 th result...


[1, 4, 9, 16, 25]

## A multi-threaded `map` function

In [ ]:
from threading import Thread

def schedule_computation_threaded(f, result, data, threads, i):
    # Each function evaluation is scheduled on a different core.
    def my_job():
        print ("Processing data:", data[i], "... ")
        result[i] = f(data[i])
        print ("Finished job #", i)
        print ("Result was", result[i])
    threads[i] = Thread(target=my_job)

def my_map_multithreaded(f, data):
    n = len(data)
    result = [None] * n
    threads = [None] * n
    print ("Scheduling jobs.. ")
    for i in range(n):
        schedule_computation_threaded(f, result, data, threads, i)
    print ("Starting jobs.. ")
    for i in range(n):
        threads[i].start()
    print ("Waiting for jobs to finish.. ")
    for i in range(n):
        threads[i].join()
    print ("All done.")
    return result

In [ ]:
my_map_multithreaded(lambda x: x*x, [1, 2, 3, 4, 5])

Scheduling jobs.. 
Starting jobs.. 
Processing data: 1 ... 
Finished job # 0
Result was 1
Processing data: 2 ... 
Finished job # 1
Result was 4
Processing data: 3 ... 
Finished job # 2
Result was 9
Processing data: 4 ... 
Finished job # 3
Result was 16
Processing data:Waiting for jobs to finish.. 
 5 ... 
Finished job # 4
Result was 25
All done.


[1, 4, 9, 16, 25]

## Map Reduce

- Map Reduce is a _programming model_ for scalable parallel processing.
- Scalable here means that it can work on big data with very large compute clusters.
- There are many implementations: e.g. Apache Hadoop and Apache Spark.
- We can use Map-Reduce with any programming language:
    - Hadoop is written in Java
    - Spark is written in Scala, but has a Python interface.
- *Functional programming* languages such as Python or Scala fit very well with the Map Reduce model:
    - However, we don't *have* to use functional programming.

## Word Count Example

- In this simple example, the input is a set of URLs, each record is a document.

- Problem: compute how many times each word has occurred across data set.

## Word Count: Map


- The input to $\operatorname{map}$ is a mapping:

- Key: URL
- Value: Contents of document

$\left< document1, to \; be \; or \; not \; to \; be \right>$  
    

- In this example, our $\operatorname{map}$ function will process a given URL, and produces a mapping:

- Key: word
- Value: 1

- So our original data-set will be transformed to:
  
  $\left< to, 1 \right>$
  $\left< be, 1 \right>$
  $\left< or, 1 \right>$
  $\left< not, 1 \right>$
  $\left< to, 1 \right>$
  $\left< be, 1 \right>$

In [ ]:
data = map(lambda x: (x, 1), "to be or not to be".split())
data

## Word Count: Shuffle

Make sure the values for the same key are grouped


Output to be equivalent to -
$\left< to, [1, 1] \right>$
  $\left< be, [1, 1] \right>$
  $\left< or, 1 \right>$
  $\left< not, 1 \right>$

In [ ]:
def groupByKey(data):
    result = dict()
    for key, value in data:
        if key in result:
            result[key].append(value)
        else:
            result[key] = [value]
    return result

In [ ]:
grouped = groupByKey(data)

## Word Count: Reduce

Output to be equivalent to -
$\left< to, 2 \right>$
  $\left< be, 2 \right>$
  $\left< or, 1 \right>$
  $\left< not, 1 \right>$
  

In [ ]:
def reduceByKey(f, data):
    key_values = groupByKey(data)
    return map(lambda key:
                   (key, reduce(f, key_values[key])),
                       key_values)

In [ ]:
reduceByKey(lambda x, y: x + y, data)